#PySpark withColumnRenamed to Rename Column on DataFrame

---

**Use PySpark withColumnRenamed() to rename a DataFrame column, we often need to rename one column or multiple (or all) columns on PySpark DataFrame, you can do this in several ways. When columns are nested it becomes complicated.**

**Since DataFrame’s are an immutable collection, you can’t rename or update a column instead when using withColumnRenamed() it creates a new DataFrame with updated column names, In this PySpark article, I will cover different ways to rename columns with several use cases like rename nested column, all columns, selected multiple columns with Python/PySpark examples.**


---


- PySpark withColumnRenamed – To rename DataFrame column name
- PySpark withColumnRenamed – To rename multiple columns
- Using StructType – To rename nested column on PySpark DataFrame
- Using Select – To rename nested columns
- Using withColumn – To rename nested columns
- Using col() function – To Dynamically rename all or multiple columns
- Using toDF() – To rename all or multiple columns


---

####First, let’s create our data set to work with.

In [0]:
dataDF = [
    (('James','','Smith'),'1991-04-01','M',3000),
  (('Michael','Rose',''),'2000-05-19','M',4000),
  (('Robert','','Williams'),'1978-09-05','M',4000),
  (('Maria','Anne','Jones'),'1967-12-01','F',4000),
  (('Jen','Mary','Brown'),'1980-02-17','F',-1)
]

**Our base schema with nested structure.**

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("name", StructType([
        StructField("firstname", StringType(), True),
        StructField("middlename", StringType(), True),
        StructField("lastname", StringType(), True)
    ])),
    StructField("dob", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("salary", IntegerType(), True)
])

**Let’s create the DataFrame by using parallelize and provide the above schema.**

In [0]:
df = spark.createDataFrame(data=dataDF, schema=schema)
df.printSchema()
df.show()

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+--------------------+----------+------+------+
|                name|       dob|gender|salary|
+--------------------+----------+------+------+
|    {James, , Smith}|1991-04-01|     M|  3000|
|   {Michael, Rose, }|2000-05-19|     M|  4000|
|{Robert, , Williams}|1978-09-05|     M|  4000|
|{Maria, Anne, Jones}|1967-12-01|     F|  4000|
|  {Jen, Mary, Brown}|1980-02-17|     F|    -1|
+--------------------+----------+------+------+



##1. PySpark withColumnRenamed – To rename DataFrame column name

---


**PySpark has a withColumnRenamed() function on DataFrame to change a column name. This is the most straight forward approach; this function takes two parameters; the first is your existing column name and the second is the new column name you wish for.**


---


####PySpark withColumnRenamed() Syntax:


#withColumnRenamed(existingName, newName)

**existingName – The existing column name you want to change**

**newName – New name of the column**

**Returns a new DataFrame with a column renamed.**

###Example

In [0]:
df.withColumnRenamed("dob", "DateOfBirth").printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



**The above statement changes column “dob” to “DateOfBirth” on PySpark DataFrame. Note that withColumnRenamed function returns a new DataFrame and doesn’t modify the current DataFrame.**

##2. PySpark withColumnRenamed – To rename multiple columns

---

**To change multiple column names, we should chain withColumnRenamed functions as shown below. You can also store all columns to rename in a list and loop through to rename all columns, I will leave this to you to explore.**

In [0]:
df2 = df.withColumnRenamed("dob", "DateOfBirth")\
.withColumnRenamed("salary", "salary_amount")

df2.printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary_amount: integer (nullable = true)



##3. Using PySpark StructType – To rename a nested column in Dataframe

---

**Changing a column name on nested data is not straight forward and we can do this by creating a new schema with new DataFrame columns using StructType and use it using cast function as shown below.**

In [0]:
schema2 = StructType([
    StructField("fname", StringType()),
    StructField("middlename", StringType()),
    StructField("lname", StringType())
])

In [0]:
from pyspark.sql.functions import col

df.select(col("name").cast(schema2),\
         col("dob"), col("gender"), col("salary"))\
.printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- fname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



**This statement renames firstname to fname and lastname to lname within name structure.**

##4. Using Select – To rename nested elements.

---

**Let’s see another way to change nested columns by transposing the structure to flat.**

In [0]:
from pyspark.sql.functions import *

df.select(col("name.firstname").alias("fname"),\
         col("name.middlename").alias("mname"),\
         col("name.lastname").alias("lname"),\
         col("dob"), col("gender"), col("salary")
         ).printSchema()

root
 |-- fname: string (nullable = true)
 |-- mname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



##5. Using PySpark DataFrame withColumn – To rename nested columns

---


**When you have nested columns on PySpark DatFrame and if you want to rename it, use withColumn on a data frame object to create a new column from an existing and we will need to drop the existing column. Below example creates a “fname” column from “name.firstname” and drops the “name” column**

In [0]:
from pyspark.sql.functions import *

df3 = df.withColumn("fname", col("name.firstname"))\
        .withColumn("mname", col("name.middlename"))\
        .withColumn("lname", col("name.lastname"))\
        .drop("name")

df3.printSchema()

root
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- fname: string (nullable = true)
 |-- mname: string (nullable = true)
 |-- lname: string (nullable = true)



##6. Using toDF() – To change all columns in a PySpark DataFrame

---

**When we have data in a flat structure (without nested) , use toDF() with a new schema to change all column names.**

In [0]:
newColumns = ["newCol1", "newCol2", "newCol3", "newCol4"]

df.toDF(*newColumns).printSchema()

root
 |-- newCol1: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- newCol2: string (nullable = true)
 |-- newCol3: string (nullable = true)
 |-- newCol4: integer (nullable = true)

